In [92]:
import pygame,os,sys
from pygame import mixer
WHITE = (255, 255, 255)
SKY_BLUE = (115,215,255)
COLOR = (252,240,151)
WIDTH = 100
HEIGHT = 100
 
# This sets the margin between each cell
MARGIN = 5

In [93]:
def load_image(path):
    image = pygame.image.load(path)
    return image

In [94]:
def get_data(path_cell, path_matrix):
    file = open(path_cell,'r')

    #Đọc hàng(row), cột(column)
    row = int(file.readline())
    column = int(file.readline())
    
    file = open(path_matrix,'r')
    lines = file.readlines()
    
    matrix = [0] * row
    for j in range (row):
        matrix[j] = [0] * column


    for i in range(column):
        element = lines[i].split(',')
        element= element[:-1]
        
        for j in range(row):
            matrix[i][j] = int(element[j])
    return row,column,matrix
a,b,c = get_data('data/size.txt','data/matrix.txt')
print(c)

[[0, 1, 0, 1], [0, 0, 2, 0], [3, 2, 0, 1], [0, 0, 0, 1]]


## Tạo 1 lớp Cell để lưu vị trí của ô hiện tại, có các hàm trả về vị trí của ô hiện tại

In [95]:
class Cell:
    def __init__(self,x,y):
        self.position_x = x
        self.position_y = y
        self.check = False
    
    def __eq__(self,other):
        return self.position_x == other.position_x and self.position_y == other.position_y
        
    def __str__(self):
        return '({}, {})'.format(self.position_x, self.position_y)
    
    def __repr__(self):
        return self.__str__()
    
    def position(self):
        return self.position_x,self.position_y

In [96]:
class Board:
    def __init__(self) -> None:
        self.core = 0
        self.spacing = 10
        #demension : kich thước
        self.cell_demension = 64
        x,y,self.matrix = get_data('data/size.txt','data/matrix.txt')
        self.matrix_demension = (x,y)

        #chiều dài, chiều rộng của phần chơi
        self.witdh = self.cell_demension * self.matrix_demension[1] + self.spacing*(self.matrix_demension[1]+1)
        self.height = self.cell_demension * self.matrix_demension[0] + self.spacing*(self.matrix_demension[0]+1)
        #vị trí của các hố
        self.holes_positions = []

        #vị trí kho báu
        self.treasure_position = []

        #vị trí quái vật
        self.wumpus_positions = []
        for row in range(self.matrix_demension[0]):
            for column in range(self.matrix_demension[1]):
                if self.matrix[column][row] == 1:
                    self.holes_positions.append(Cell(row,column))
                if self.matrix[column][row] == 2:
                    self.wumpus_positions.append(Cell(row,column))
                if self.matrix[column][row] == 3:
                    self.treasure_position.append(Cell(row,column))

a = Board()
print(a.holes_positions[0].position()[0])

1


In [97]:
class Holes:
    def __init__(self,board) -> None:
        self.board = board
        self.hole = 'images/hole/hole.png'
        self.breeze = 'images/hole/breeze-1.png'
        self.visit = False
        self.breeze_visit = False
        self.breeze_distance = 1
        self.breeze_position = []
        
    def breeze_hole(self):
        
        for i in range(len(self.board.holes_positions)):
            breeze = self.board.holes_positions[i]
            if Cell(breeze.position()[0]+1,breeze.position()[1]) not in self.breeze_position and breeze.position()[0] <3 and Cell(breeze.position()[0]+1,breeze.position()[1]) not in self.board.holes_positions:
                    self.breeze_position.append(Cell(breeze.position()[0]+1,breeze.position()[1]))
            if Cell(breeze.position()[0]-1,breeze.position()[1]) not in self.breeze_position and breeze.position()[0] > 0 and Cell(breeze.position()[0]-1,breeze.position()[1]) not in self.board.holes_positions:
                    self.breeze_position.append(Cell(breeze.position()[0]-1,breeze.position()[1]))
            if Cell(breeze.position()[0],breeze.position()[1]+1) not in self.breeze_position and breeze.position()[1] <3 and Cell(breeze.position()[0],breeze.position()[1]+1) not in self.board.holes_positions:
                    self.breeze_position.append(Cell(breeze.position()[0],breeze.position()[1]+1))
            if Cell(breeze.position()[0],breeze.position()[1]-1) not in self.breeze_position and breeze.position()[1] >0 and Cell(breeze.position()[0],breeze.position()[1]-1) not in self.board.holes_positions:
                    self.breeze_position.append(Cell(breeze.position()[0],breeze.position()[1]-1))
        return self.breeze_position
    
b = Holes(a)
print(b.breeze_hole())
print(Cell(1,1) not in b.breeze_hole())


[(2, 0), (0, 0), (1, 1), (3, 1), (2, 2), (2, 3)]
False


In [98]:
class Wumpus():
    def __init__(self,board) -> None:
        self.board = board
        self.visit = False
        self.live = True
        self.wumpus = 'images/wumpus/wumpus.png'
        self.smell = 'images/wumpus/smell-1.png'
        self.smell_position = []
        self.smell_visible = False

    def smell_wumpus(self):
        for i in range(len(self.board.wumpus_positions)):
            smell = self.board.wumpus_positions[i]
            if Cell(smell.position()[0]+1,smell.position()[1]) not in self.smell_position and smell.position()[0] <3 and Cell(smell.position()[0]+1,smell.position()[1]) not in self.board.wumpus_positions:
                    self.smell_position.append(Cell(smell.position()[0]+1,smell.position()[1]))
            if Cell(smell.position()[0]-1,smell.position()[1]) not in self.smell_position and smell.position()[0] > 0 and Cell(smell.position()[0]-1,smell.position()[1]) not in self.board.wumpus_positions:
                    self.smell_position.append(Cell(smell.position()[0]-1,smell.position()[1]))
            if Cell(smell.position()[0],smell.position()[1]+1) not in self.smell_position and smell.position()[1] <3 and Cell(smell.position()[0],smell.position()[1]+1) not in self.board.wumpus_positions:
                    self.smell_position.append(Cell(smell.position()[0],smell.position()[1]+1))
            if Cell(smell.position()[0],smell.position()[1]-1) not in self.smell_position and smell.position()[1] >0 and Cell(smell.position()[0],smell.position()[1]-1) not in self.board.wumpus_positions:
                    self.smell_position.append(Cell(smell.position()[0],smell.position()[1]-1))
        return self.smell_position

b = Wumpus(a)


In [99]:
class Treasure():
    def __init__(self,board) -> None:
        self.board = board
        self.visit = False
        self.image = 'images/treasure.png'
        self.treasure = self.board.treasure_position

c = Treasure(a)
print(c.treasure)

[(0, 2)]


In [100]:
class Hunter():
    def __init__(self,board) -> None:
        self.board = board
        self.position_x = 0
        self.position_y = 0
        self.count_arrow = 0

        self.moved = False
        self.arrow = True

        self.direction = "RIGHT"
        self.image = 'images/hunter/hunter_right.png'

    def increment_x(self):
        self.image = "images/hunter/hunter_right.png"
        self.moved = False

        if self.direction == "RIGHT":
            self.position_x += 1
            self.moved = True
        self.direction = "RIGHT"
    
    def decrement_x(self):
        self.image = "images/hunter/hunter_left.png"
        self.moved = False

        if self.direction == "LEFT":
            self.position_x -= 1
            self.moved = True
        self.direction = "LEFT"
    
    def increment_y(self):
        self.image = "images/hunter/hunter_down.png"
        self.moved = False

        if self.direction == "DOWN":
            self.position_y += 1
            self.moved = True
        self.direction = "DOWN"

    def decrement_y(self):
        self.image = "images/hunter/hunter_up.png"
        self.moved = False

        if self.direction == "UP":
            self.position_y -= 1
            self.moved = True
        self.direction = "UP"

    def arrow_direction(self):
        if self.direction == "DOWN":
            return "images/arrow/arrow_down.png"
        elif self.direction == "UP":
            return "images/arrow/arrow_up.png"
        elif self.direction == "RIGHT":
            return "images/arrow/arrow_right.png"
        elif self.direction == "LEFT":
            return "images/arrow/arrow_left.png"
    
    def arrow_position(self, distance=1):
        if self.direction == "DOWN":
            return (self.position_x, self.position_y + distance)
        elif self.direction == "UP":
            return (self.position_x, self.position_y - distance)
        elif self.direction == "RIGHT":
            return (self.position_x + distance, self.position_y)
        elif self.direction == "LEFT":
            return (self.position_x - distance, self.position_y)
        
    def __str__(self):
        return '({}, {})'.format(self.position_x, self.position_y)

    def __repr__(self):
        return self.__str__()

    def position(self):
        return self.position_x, self.position_y

b =Hunter(a)
print(b.arrow_position())
print(b.direction)
# print(b.arrow_direction())
print(b.position())

b.increment_x()
print(b.direction)
print(b.arrow_position())
# print(b.arrow_direction())
print(b.position())


(1, 0)
RIGHT
(0, 0)
RIGHT
(2, 0)
(1, 0)


In [101]:
def draw_font(text,size,color,path):
    return pygame.font.Font(path,size).render(text,False,color)

In [102]:

running = True

game_over = False
# This is a font we use to draw text on the screen (size 36)
winner = False
def play():
    pygame.init()
    
    board = Board()
    hunter = Hunter(board)
    wumpus = Wumpus(board)
    holes = Holes(board)
    treasure = Treasure(board)
    
    cell = load_image('images/cell/wall.png')
    font = pygame.font.Font('fonts/mrsmonster.ttf',50)
    screentype = 0
    screen = pygame.display.set_mode((600,400),screentype,32)
    text = font.render('Wumpus World',False,COLOR)
    pygame.display.set_caption("Wumpus World")
    clock = pygame.time.Clock()
    
    
    def draw_hunter():
        x = (hunter.position_x * board.cell_demension) + (hunter.position_x * board.spacing) + board.spacing
        y = (hunter.position_y * board.cell_demension) + (hunter.position_y * board.spacing) + board.spacing
        hunter_image = load_image(hunter.image)
        screen.blit(hunter_image, (x+board.cell_demension, y+board.cell_demension))

    def draw_wumpus_smells():
        wumpus_smell_image = load_image(wumpus.smell)
        wumpus_smells = wumpus.smell_wumpus()
        for c in wumpus_smells:
            if wumpus.smell_visible or Cell(c.position()[0],c.position()[1]).check:
                x = (c.position()[0] * board.cell_demension) + (c.position()[0] * board.spacing) + board.spacing
                y = (c.position()[1] * board.cell_demension) + (c.position()[1] * board.spacing) + board.spacing
                screen.blit(wumpus_smell_image, (x+board.cell_demension, y+board.cell_demension))

    def draw_wumpus():
        wumpus_image = load_image(wumpus.wumpus)
        for c in board.wumpus_positions:
            if wumpus.visit or Cell(c.position()[0],c.position()[1]).check:
                x = (c.position()[0] * board.cell_demension) + (c.position()[0] * board.spacing) + board.spacing
                y = (c.position()[1] * board.cell_demension) + (c.position()[1]* board.spacing) + board.spacing
                
                screen.blit(wumpus_image, (x+board.cell_demension, y+board.cell_demension))

        draw_wumpus_smells()
    
    def draw_hole_breeze():
        hole_breeze = load_image(holes.breeze)
        hole_breezes = holes.breeze_hole()
        for c in hole_breezes:
            if wumpus.smell_visible or Cell(c.position()[0],c.position()[1]).check:
                x = (c.position()[0] * board.cell_demension) + (c.position()[0] * board.spacing) + board.spacing
                y = (c.position()[1] * board.cell_demension) + (c.position()[1] * board.spacing) + board.spacing
                screen.blit(hole_breeze, (x+board.cell_demension, y+board.cell_demension))

    def draw_hole():
        hole_image = load_image(holes.hole)
        for c in board.holes_positions:
            if holes.visit or Cell(c.position()[0],c.position()[1]).check:
                x = (c.position()[0] * board.cell_demension) + (c.position()[0] * board.spacing) + board.spacing
                y = (c.position()[1] * board.cell_demension) + (c.position()[1]* board.spacing) + board.spacing
                
                screen.blit(hole_image, (x+board.cell_demension, y+board.cell_demension))

        draw_hole_breeze()

    def draw_treasure():
        treasure_image = load_image(treasure.image)
        for c in board.treasure_position:
            if treasure.visit or Cell(c.position()[0],c.position()[1]).check:
                x = (c.position()[0] * board.cell_demension) + (c.position()[0] * board.spacing) + board.spacing
                y = (c.position()[1] * board.cell_demension) + (c.position()[1] * board.spacing) + board.spacing
                screen.blit(treasure_image, (x+board.cell_demension, y+board.cell_demension))


    def present_wumpus(position):
        global game_over
        game_over = True
        x = (position[0] * board.cell_demension) + (position[0] * board.spacing) + board.spacing
        y = (position[1] * board.cell_demension) + (position[1] * board.spacing) + board.spacing

        wumpus_image = load_image(wumpus.wumpus)
        screen.blit(wumpus_image, (x+ board.cell_demension, y+board.cell_demension))

    def present_smell(position):
        x = (position[0] * board.cell_demension) + (position[0] * board.spacing) + board.spacing
        y = (position[1] * board.cell_demension) + (position[1] * board.spacing) + board.spacing

        wumpus_smell = load_image(wumpus.smell)
        screen.blit(wumpus_smell, (x+ board.cell_demension, y+ board.cell_demension))

    def present_wumpus_arrow(position):
        x = (position[0] * board.cell_demension) + (position[0] * board.spacing) + board.spacing
        y = (position[1] * board.cell_demension) + (position[1] * board.spacing) + board.spacing

        wumpus_image = load_image(wumpus.wumpus)
        screen.blit(wumpus_image, (x+ board.cell_demension, y+ board.cell_demension))

    def present_treasure(position):
        global winner
        winner = True

        x = (position[0] * board.cell_demension) + (position[0] * board.spacing) + board.spacing
        y = (position[1] * board.cell_demension) + (position[1] * board.spacing) + board.spacing

        treasure_image = load_image(treasure.image)
        screen.blit(treasure_image, (x+board.cell_demension, y+board.cell_demension))

    def present_breeze(position):
        x = (position[0] * board.cell_demension) + (position[0] * board.spacing) + board.spacing
        y = (position[1] * board.cell_demension) + (position[1] * board.spacing) + board.spacing

        breeze_image = load_image(holes.breeze)
        screen.blit(breeze_image, (x+board.cell_demension, y+board.cell_demension))


    def present_holes(position):
        global game_over
        game_over = True

        x = (position[0] * board.cell_demension) + (position[0] * board.spacing) + board.spacing
        y = (position[1] * board.cell_demension) + (position[1] * board.spacing) + board.spacing

        holes_image = load_image(holes.hole)
        screen.blit(holes_image, (x+board.cell_demension, y+board.cell_demension))


    def present():
        hunter_position = Cell(hunter.position()[0],hunter.position()[1])
        hunter_position_1 = hunter.position()
        smells_position = wumpus.smell_position
        breezes_position = holes.breeze_position

        if hunter_position in wumpus.board.wumpus_positions :
            if wumpus.live:
                present_wumpus(hunter_position_1)
            else:
                present_smell(hunter_position_1)

        if hunter_position in treasure.board.treasure_position:
            present_treasure(hunter_position_1)

        if hunter_position in smells_position:
            present_smell(hunter_position_1)

        if hunter_position in breezes_position:
            present_breeze(hunter_position_1)

        if hunter_position in holes.board.holes_positions:
            present_holes(hunter_position_1)

    def match_result():
        global running,game_over,winner
        if game_over:
            mixer.music.load("musics/audio_game-over.wav")
            mixer.music.play()
            text = font.render("You Lose!", True, (153, 0, 0))
            text_rect = text.get_rect()
            text_x = board.witdh / 2 - text_rect.width / 2
            text_y = board.height / 2 - text_rect.height / 2
            screen.blit(text, [text_x, text_y])
            running = False
            game_over = False
            
            
            

        if winner:
            mixer.music.load("musics/audio_win.wav")
            mixer.music.play()
            text = font.render("You Win!", True, (0, 0, 255))
            text_rect = text.get_rect()
            text_x = board.witdh / 2 - text_rect.width / 2
            text_y = board.height / 2 - text_rect.height / 2
            screen.blit(text, [text_x, text_y])
            running = False
            winner = False
            
            
            

    def killer_wumpus_message():
        text = font.render("Wumpus died!", True, (153, 0, 0))
        text_rect = text.get_rect()
        text_x = board.witdh / 2 - text_rect.width / 2
        text_y = board.height / 2 - text_rect.height / 2
        screen.blit(text, [text_x, text_y])

    def draw_matrix():
        x = board.spacing
        y = board.spacing
        cell1 = load_image('images/cell/wall_1.png')

        for i in range(0, board.matrix_demension[0]):
            for j in range(0, board.matrix_demension[1]):
                # if not Cell(i,j).check:
                screen.blit(cell, (x+board.cell_demension, y+board.cell_demension))
                # else:
                    # screen.blit(cell1, (x+board.cell_demension, y+board.cell_demension))
                x += (board.spacing + board.cell_demension)
            y += (board.spacing + board.cell_demension)
            x = board.spacing

        draw_hole()
        draw_treasure()
        present()
        draw_hunter()
        match_result()
        if(wumpus.live):
            draw_wumpus()
        

        pygame.display.update()
    
    def hunter_move_left():
        if hunter.position_x > 0:
            hunter.decrement_x()
            draw_matrix()


    def hunter_move_right():
        if hunter.position_x < board.matrix_demension[1] - 1:
            hunter.increment_x()
            draw_matrix()


    def hunter_move_up():
        if hunter.position_y > 0:
            hunter.decrement_y()
            draw_matrix()


    def hunter_move_down():
        if hunter.position_y < board.matrix_demension[0] - 1:
            hunter.increment_y()
            draw_matrix()

    def wumpus_died():
        killer_wumpus_message()
        wumpus.live = False
        
        pygame.display.update()
        pygame.time.wait(2000)
        
        init()

    def throw_arrow():
        sound = mixer.Sound('musics/audio_arrow.wav')
        sound.play()
        arrow_image = load_image(hunter.arrow_direction())
            
        x_arrow = hunter.arrow_position()[0]
        y_arrow = hunter.arrow_position()[1]
        arrow_position = (x_arrow, y_arrow)

        x = (x_arrow * board.cell_demension) + (x_arrow * board.spacing) + board.spacing
        y = (y_arrow * board.cell_demension) + (y_arrow * board.spacing) + board.spacing

        cell_arrow = Cell(x_arrow,y_arrow)

        if cell_arrow in board.wumpus_positions:
            present_wumpus_arrow(arrow_position)
            screen.blit(arrow_image, (x+board.cell_demension, y+board.cell_demension))
            wumpus_died()
        else:
            screen.blit(arrow_image, (x+board.cell_demension, y+board.cell_demension))
            pygame.display.update()
            pygame.time.wait(500)
            screen.blit(cell, (x+board.cell_demension, y+board.cell_demension))
        hunter.count_arrow += 1
    
    def debug_mode():
        wumpus.visit= not wumpus.visit
        wumpus.smell_visible = not wumpus.smell_visible
        treasure.visit = not treasure.visit
        holes.visit = not holes.visit
        holes.breeze_visit = not holes.breeze_visit
        draw_matrix()

    def init():
        mixer.music.load("musics/background.mp3")
        mixer.music.play()
        screen.fill(SKY_BLUE)
        screen.blit(text,(120,0))
        draw_matrix()
        run()

    def run():
        global running
        running = True
        while running:
            clock.tick(60)
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                    pygame.quit()
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_LEFT:
                        hunter_move_left()
                        
                    elif event.key == pygame.K_RIGHT:
                        hunter_move_right()

                    elif event.key == pygame.K_UP:
                        hunter_move_up()

                    elif event.key == pygame.K_DOWN:
                        hunter_move_down()
                        
                    elif event.key == pygame.K_SPACE:
                        if(hunter.arrow):
                            throw_arrow()
                    elif event.key == pygame.K_d:
                        debug_mode()
            
#             pygame.display.update()
        pygame.time.wait(4000)
        
    init()

In [103]:
def run_menu():
    pygame.init()

    background = load_image('images/background.png')
    button_play = load_image('images/button/button-play.png')
    button_import = load_image('images/button/button-import.png')
    button_music = load_image('images/button/button-music.png')
    button_quit = load_image('images/button/button-quit.png')

    screentype = 0
    screen = pygame.display.set_mode((600,400),screentype,32)
    pygame.display.set_caption("Wumpus World")
    clock = pygame.time.Clock()
    mixer.music.set_volume(0)
    running = True
    while running:
        mousex, mousey = pygame.mouse.get_pos()
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                pygame.quit()
                sys.exit(1)
            if event.type == pygame.MOUSEBUTTONDOWN:
                # Tạo action button quit
                if 330<mousey<370 and 220<mousex<340:  
                    print("Đã nhấn")
                    pygame.quit()
                    sys.exit(1)

                #Tạo action button play
                if 120<mousey<165 and 220<mousex<340:
                    play()
                #Tạo action button music
                if 190<mousey<235 and 220<mousex<340:
                    print("Đã nhấn Music")
                    
                #Tạo action button import, import từ file data ngoài vào
                if 260<mousey<305 and 220<mousex<340:
                    mixer.music.set_volume(1)
                
        screen.blit(background,(0,0))
        screen.blit(button_play,(200,110))
        screen.blit(button_import,(200,180))
        screen.blit(button_music,(200,250))
        screen.blit(button_quit,(200,320))
        pygame.display.update()
        clock.tick(60)
    sys.exit(1)

In [104]:
run_menu()

error: display Surface quit